# Practicum HW5. H+, or how to build a perfect human.

The project can be effectively boiled down to following tasks:
- "Where do we come from?" Establish maternal (mtDNA) and paternal (Y-chromosome) haplogroups and, optionally,  probable ethnicity.
- "Who are we?" Annotate the obtaining SNPs and extract all clinically relevant SNPs from the ClinVar database. If you succeed, you will immediately become a valuable specialist in the job market (if you haven't already forgotten how to make a vcf file from the raw reads). Identify phenotypic traits from the raw data ("genome sketching"). 
- "Where are we going?" Make changes to a given genome to get a person with the desired characteristics. I.e. find, say, 5 variants that would be useful, and specify what nucleotide exactly and at what position in this genome needs to be replaced using CRISPR.


### 1. Dataset

23andMe: https://drive.google.com/open?id=1QJkwJe5Xl_jSVpqdTSNXP7sqlYfI666j

Genotek: https://drive.google.com/file/d/1piKi0FxVao-IvG81TAXhDKlHXvHE6uTm/view?usp=sharing

#### Preprocessing

First, remove all SNPs corresponding to deletions and insertions to make the file compatible with annotation tools.
``` 
plink --23file data/23_and_me_input.txt \
      --recode vcf \
      --out data/snps_clean \
      --output-chr MT \
      --snps-only just-acgt
```

The resulting file contains all the analyzed SNPs, and we are interested only in variable positions.


#### Origins, haplogroups.


Establish probable ethnicity of given subject by identifying maternal (mtDNA) and paternal (Y chromosome) haplogroups. 

There are many ready-made tools from colleagues at the International Society of Genetic Genealogy - https://isogg.org/wiki/MtDNA_tools 

For example, for mtDNA https://dna.jameslick.com/mthap/ - shows all SNPs that distinguish the haplogroup, and takes 23andMe input.
![Jameslick output](imgs/jameslick.png)

For the Y chromosome there is also a lot of interesting stuff - e.g.  https://isogg.org/wiki/Y-DNA_tools

For raw 23andMe, for example, this one: https://ytree.morleydna.com/extractFromAutosomal
![Jameslick output](imgs/morley.png)


#### Annotation - sex and eye colour

![SNPs for eyes color and skin tone](imgs/eyes_skin_SNPs.png)


rs12913832 A/A |\
rs16891982 C/C |\
rs12896399 G/G | -> Brown eyes \
rs6119471 -    |\
rs12203592 C/C |




#### Annotation of all SNPs, selection of clinically relevant ones.

1. snpEff - functional annotation. Not very interesting here 
    ```
    snpEff GRCh37.75 data/snps_clean.vcf  > data/snps_clean.snpeff.vcf
    ```
    Conda installed snpEff gets Java heap memmory leak, better use

    ```
    java -Xmx20g -jar ../../snpEff/snpEff.jar GRCh37.75 data/snps_clean.vcf  > data/snps_clean.snpeff.vcf
    ```

2. SnpSift - phenotypic annotation
    - index vfc.gz
    ```
    tabix -p vcf data/clinvar.vcf.gz
    ```
    - annotate
    ```
    SnpSift annotate data/clinvar.vcf.gz  data/snps_clean.vcf > data/snps_clean.snpsift_clinvar.vcf
    ```

    2.1. Look at CLNDN
        ```
        cat data/snps_clean.snpsift_clinvar.vcf | grep CLNDN | grep -v 'CLNDN=not_provided'
        ```

        - got 864 rows where CLNDN is not 'not_provided'

3. VEP (Variant Effect Predictor)

![Example of VEP output](imgs/VEP.png)


1. COMT gene (rs4680): The COMT gene codes for the COMT enzyme, which breaks down dopamine in the brain's prefrontal cortex. The wild-type allele is a (G), coding for a valine amino acid; the (A) substitution polymorphism changes the amino acid to a methionine. This alters the structure of the resultant enzyme such that its activity is only 25% of the wild type. As a result, A allele carriers have more dopamine in their prefrontal cortex, which may be responsible for many of the neuropsychological associations.

    rs4680(A) = Worrier. Met, more exploratory, lower COMT enzymatic activity, therefore higher dopamine levels; lower pain threshold, enhanced vulnerability to stress, yet also more efficient at processing information under most conditions

    **rs4680(G)** = Warrior. Val, less exploratory, higher COMT enzymatic activity, therefore lower dopamine levels; higher pain threshold, better stress resiliency, albeit with a modest reduction in executive cognition performance under most conditions

2. ABO blood group system (rs8176746): Determines blood type and has been linked to various health outcomes, including susceptibility to certain infections and cardiovascular disease.

    rs8176746(T) has a status 'affects' in clinvar.

3. BDNF gene (rs6265): This SNP is associated with brain-derived neurotrophic factor, which plays a role in neuronal growth and survival. Variants in this gene have been linked to differences in cognitive function, mood disorders, and susceptibility to psychiatric conditions like depression and anxiety. Variant is marked as benign and risk factor in clinvar, although there were no clear evidence in the reference article.

4. SLC2A9 gene (rs16890979): Linked to uric acid metabolism and risk of gout. Variants in this gene have been associated with differences in serum urate levels and susceptibility to gout. rs16890979 has been found to be associated with gout in several independent studies. It may be a variation in the SLC2A9 gene, which is more commonly known as GLUT9. So here we have a binign variant that is associated with 1.7x risk of gout according to SNPedia.

